In [21]:
# Imports
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import flwr as fl
from typing import Dict, List, Tuple
import numpy as np
from collections import OrderedDict
import os
from pathlib import Path
import tempfile
import logging
import json
from datetime import datetime
import time

In [22]:
# Set client ID (change for each client: 1,2,3,4)
CLIENT_ID = 1  # Change this for each client

In [23]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(f"Client_{CLIENT_ID}")

def log_status(status: str, details: str = ""):
    status_line = f"\n{'='*20} {status} {'='*20}"
    logger.info(status_line)
    if details:
        logger.info(details)
    logger.info("="*len(status_line))
    print(status_line)
    if details:
        print(details)
    print("="*len(status_line))

# Set random seed
torch.manual_seed(42 + CLIENT_ID)

# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_status("DEVICE INFO", f"Client {CLIENT_ID} using device: {device}")

2025-03-09 15:13:32,724 - Client_1 - INFO - 
==================== DEVICE INFO ====================
2025-03-09 15:13:32,725 - Client_1 - INFO - Client 1 using device: cuda
2025-03-09 15:13:32,725 - Client_1 - INFO - ======================================================



==================== DEVICE INFO ====================
Client 1 using device: cuda


In [24]:
# Create temporary directory for outputs
temp_dir = tempfile.mkdtemp()
log_status("DIRECTORY INFO", f"Using temporary directory: {temp_dir}")

# Define message size
GRPC_MAX_MESSAGE_LENGTH = 1024 * 1024 * 1024

2025-03-09 15:13:32,754 - Client_1 - INFO - 
==================== DIRECTORY INFO ====================
2025-03-09 15:13:32,755 - Client_1 - INFO - Using temporary directory: C:\Users\NILATA~1\AppData\Local\Temp\tmp9bgt6qn4
2025-03-09 15:13:32,755 - Client_1 - INFO - =========================================================



==================== DIRECTORY INFO ====================
Using temporary directory: C:\Users\NILATA~1\AppData\Local\Temp\tmp9bgt6qn4


In [25]:
# Load dataset
log_status("LOADING DATASET", f"Client {CLIENT_ID} loading dataset...")
try:
    dataset = load_dataset("medalpaca/medical_meadow_medical_flashcards")
    
    # Select different ranges for each client
    start_idx = (CLIENT_ID - 1) * 1000
    end_idx = CLIENT_ID * 1000
    small_dataset = dataset['train'].select(range(start_idx, end_idx))
    
    log_status("DATASET LOADED", 
              f"Dataset size: {len(small_dataset)} examples\n"
              f"Range: {start_idx} to {end_idx}")
    
    # Extract test questions from the dataset
    test_indices = np.linspace(0, len(small_dataset)-1, 5, dtype=int)

    test_questions = []
    test_answers = []
    for idx in test_indices:
        example = small_dataset[int(idx)]  # Convert idx to Python int
        test_questions.append(example['input'])   # Ensure correct column names
        test_answers.append(example['output'])    # Ensure correct column names
    
    log_status("TEST QUESTIONS SELECTED", 
              f"Number of test questions: {len(test_questions)}")
    
except Exception as e:
    log_status("DATASET ERROR", str(e))
    raise

2025-03-09 15:13:32,780 - Client_1 - INFO - 
==================== LOADING DATASET ====================
2025-03-09 15:13:32,782 - Client_1 - INFO - Client 1 loading dataset...
2025-03-09 15:13:32,782 - Client_1 - INFO - ==========================================================



==================== LOADING DATASET ====================
Client 1 loading dataset...


2025-03-09 15:13:34,540 - Client_1 - INFO - 
==================== DATASET LOADED ====================
2025-03-09 15:13:34,542 - Client_1 - INFO - Dataset size: 1000 examples
Range: 0 to 1000
2025-03-09 15:13:34,543 - Client_1 - INFO - =========================================================
2025-03-09 15:13:34,544 - Client_1 - INFO - 
==================== TEST QUESTIONS SELECTED ====================
2025-03-09 15:13:34,544 - Client_1 - INFO - Number of test questions: 5
2025-03-09 15:13:34,545 - Client_1 - INFO - ==================================================================



==================== DATASET LOADED ====================
Dataset size: 1000 examples
Range: 0 to 1000

==================== TEST QUESTIONS SELECTED ====================
Number of test questions: 5


In [26]:
test_questions

['What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?',
 'Can you describe Gardner syndrome and the conditions that it is associated with?',
 'What is Plummer-Vinson syndrome and what are the main symptoms associated with this condition?',
 'What is the genetic cause of cystic fibrosis, and which gene and chromosome is affected by this defect?',
 'What is the relationship between upper motoneuron lesions and paralysis?']

In [27]:
# Format flashcards
def format_flashcard(example):
    return {
        'text': f"Question: {example['input']}\nAnswer: {example['output']}\n\n"
    }

formatted_dataset = small_dataset.map(format_flashcard)

c:\Users\Nil Atabey\anaconda3\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
# Initialize model and tokenizer
log_status("MODEL INITIALIZATION", "Loading model and tokenizer...")
try:
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    
    # Configure tokenizer
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id
    
    log_status("MODEL LOADED", 
              f"Model: {model_name}\n"
              f"Parameters: {sum(p.numel() for p in model.parameters())}")
except Exception as e:
    log_status("MODEL ERROR", str(e))
    raise

2025-03-09 15:13:34,774 - Client_1 - INFO - 
==================== MODEL INITIALIZATION ====================
2025-03-09 15:13:34,776 - Client_1 - INFO - Loading model and tokenizer...
2025-03-09 15:13:34,776 - Client_1 - INFO - ===============================================================



==================== MODEL INITIALIZATION ====================
Loading model and tokenizer...


2025-03-09 15:13:36,528 - Client_1 - INFO - 
==================== MODEL LOADED ====================
2025-03-09 15:13:36,529 - Client_1 - INFO - Model: gpt2
Parameters: 124439808
2025-03-09 15:13:36,529 - Client_1 - INFO - =======================================================



==================== MODEL LOADED ====================
Model: gpt2
Parameters: 124439808


In [29]:
# Function to generate answers
def generate_answer(question: str, max_length: int = 100) -> str:
    try:
        prompt = f"Question: {question}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            temperature=0.7
        )
        
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        logger.error(f"Error generating answer: {e}")
        return f"Error generating answer: {str(e)}"

In [30]:
# Function to evaluate model responses
def evaluate_model_responses(phase="Before"):
    log_status(f"{phase.upper()} TRAINING EVALUATION", "Starting model evaluation...")
    responses = {}
    for q, a in zip(test_questions, test_answers):
        response = generate_answer(q)
        responses[q] = {
            'model_response': response,
            'ground_truth': a
        }
        print(f"\nQuestion: {q}")
        print(f"Model Response: {response}")
        print(f"Ground Truth: {a}")
        print("-" * 50)
    return responses

In [31]:
# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )

log_status("TOKENIZATION", "Tokenizing dataset...")
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    remove_columns=formatted_dataset.column_names,
    batched=True
)

2025-03-09 15:13:36,588 - Client_1 - INFO - 
==================== TOKENIZATION ====================
2025-03-09 15:13:36,589 - Client_1 - INFO - Tokenizing dataset...
2025-03-09 15:13:36,590 - Client_1 - INFO - =======================================================



==================== TOKENIZATION ====================
Tokenizing dataset...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [32]:
# Training arguments
training_args = TrainingArguments(
    output_dir=os.path.join(temp_dir, f"client_{CLIENT_ID}"),
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=5e-5,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",
    save_total_limit=2,
    overwrite_output_dir=True,
)

c:\Users\Nil Atabey\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [34]:
# Define Flower client
class MedicalFlashcardsClient(fl.client.NumPyClient):
    def __init__(self):
        log_status("CLIENT INITIALIZATION", f"Client {CLIENT_ID} initializing...")
        self.trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
            data_collator=data_collator,
        )
        log_status("CLIENT READY", f"Client {CLIENT_ID} initialized and ready")
        
    def get_parameters(self, config: Dict[str, str]) -> List[np.ndarray]:
        log_status("PARAMETER RETRIEVAL", f"Client {CLIENT_ID}: Getting parameters")
        return [val.cpu().numpy() for _, val in model.state_dict().items()]
    
    def set_parameters(self, parameters: List[np.ndarray]) -> None:
        log_status("PARAMETER UPDATE", f"Client {CLIENT_ID}: Setting parameters")
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)
    
    def fit(
        self, parameters: List[np.ndarray], config: Dict[str, str]
    ) -> Tuple[List[np.ndarray], int, Dict[str, float]]:
        log_status("TRAINING START", f"Client {CLIENT_ID}: Starting training round")
        self.set_parameters(parameters)
        self.trainer.train()
        log_status("TRAINING COMPLETE", f"Client {CLIENT_ID}: Completed training round")
        return self.get_parameters(config), len(tokenized_dataset), {}
    
    def evaluate(
        self, parameters: List[np.ndarray], config: Dict[str, str]
    ) -> Tuple[float, int, Dict[str, float]]:
        log_status("EVALUATION", f"Client {CLIENT_ID}: Evaluating model")
        self.set_parameters(parameters)
        metrics = self.trainer.evaluate()
        return float(metrics["eval_loss"]), len(tokenized_dataset), {"loss": float(metrics["eval_loss"])}


In [35]:
# Save model function
def save_model(path: str = None):
    try:
        if path is None:
            path = os.path.join(temp_dir, f"medical-model-client-{CLIENT_ID}")
        
        Path(path).mkdir(parents=True, exist_ok=True)
        model.save_pretrained(path)
        tokenizer.save_pretrained(path)
        log_status("MODEL SAVED", f"Model saved to {path}")
    except Exception as e:
        log_status("SAVE ERROR", str(e))
        # Try fallback location
        home_dir = os.path.expanduser("~")
        fallback_path = os.path.join(home_dir, f"medical_model_backup_client_{CLIENT_ID}")
        Path(fallback_path).mkdir(parents=True, exist_ok=True)
        model.save_pretrained(fallback_path)
        tokenizer.save_pretrained(fallback_path)
        log_status("FALLBACK SAVE", f"Model saved to fallback location: {fallback_path}")

In [36]:
# Calculate similarity between responses
def calculate_similarity(str1, str2):
    words1 = set(str1.lower().split())
    words2 = set(str2.lower().split())
    overlap = len(words1.intersection(words2))
    union = len(words1.union(words2))
    return overlap / union if union > 0 else 0

# Test before training
log_status("PRE-TRAINING EVALUATION", "Testing model before training...")
before_responses = evaluate_model_responses("Before")

2025-03-09 15:13:36,948 - Client_1 - INFO - 
==================== PRE-TRAINING EVALUATION ====================
2025-03-09 15:13:36,948 - Client_1 - INFO - Testing model before training...
2025-03-09 15:13:36,950 - Client_1 - INFO - ==================================================================
2025-03-09 15:13:36,951 - Client_1 - INFO - 
==================== BEFORE TRAINING EVALUATION ====================
2025-03-09 15:13:36,951 - Client_1 - INFO - Starting model evaluation...
2025-03-09 15:13:36,951 - Client_1 - INFO - =====================================================================
c:\Users\Nil Atabey\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may obse


==================== PRE-TRAINING EVALUATION ====================
Testing model before training...

==================== BEFORE TRAINING EVALUATION ====================
Starting model evaluation...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Model Response: Question: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Answer: The relationship is not clear. The most common explanation is that the higher the M g2 level, the more Ca 2+ is present. However, this is a very difficult question to answer.
The most commonly accepted explanation for the correlation between M G2 and M Th is:
Mg 2 = M + M (M + Th)
This
Ground Truth: Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.
--------------------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question: Can you describe Gardner syndrome and the conditions that it is associated with?
Model Response: Question: Can you describe Gardner syndrome and the conditions that it is associated with?
Answer: Gardner Syndrome is a condition that occurs when a person is unable to communicate with others. It is characterized by a lack of communication and inability to understand the meaning of words.
Gardner Syndrome can be diagnosed by looking at the eyes, nose, or throat. The symptoms of Gardner are:

A lack or inability of coordination
 (a lack in coordination)
, (A inability or unwillingness
Ground Truth: Gardner syndrome is a medical condition that is associated with the combination of familial adenomatous polyposis, osteomas, and fibromatosis.
--------------------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question: What is Plummer-Vinson syndrome and what are the main symptoms associated with this condition?
Model Response: Question: What is Plummer-Vinson syndrome and what are the main symptoms associated with this condition?
Answer: Plum-vinson is a condition that occurs when the body's immune system is weakened. It is caused by a deficiency of the immune response to the placenta. The plum is the part of your body that is responsible for producing the antibodies that protect your baby from the infection.
Plum vinson can be caused either by the mother's own immune deficiency or
Ground Truth: Plummer-Vinson syndrome is a medical condition characterized by iron deficiency anemia, esophageal webs, and atrophic glossitis. Some common symptoms of this syndrome include difficulty swallowing, mouth sores, and fatigue.
--------------------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question: What is the genetic cause of cystic fibrosis, and which gene and chromosome is affected by this defect?
Model Response: Question: What is the genetic cause of cystic fibrosis, and which gene and chromosome is affected by this defect?
Answer: The genetic causes of the cyst are not known. However, the most common cause is a genetic mutation in the gene for the enzyme that causes the fibroblast cell to grow. The gene is responsible for producing the protein that makes cysts.
The cysteine is an enzyme in cytoplasm that converts cytochrome P450 (CYP
Ground Truth: Cystic fibrosis is caused by a genetic defect in the CFTR gene, which is located on chromosome 7. This gene is responsible for producing a protein that regulates the movement of salt and water in and out of cells. When the CFTR gene is defective, it leads to the production of thick, sticky mucus in the lungs, pancreas, and other organs, which can cause a range of symptoms and complications associated with cystic fibrosis

In [37]:
# Start Flower client
log_status("CONNECTION SETUP", 
         f"Starting Flower client {CLIENT_ID}\n"
         f"Server address: 127.0.0.1:8081")

connection_attempts = 0
max_attempts = 3
retry_delay = 5

while connection_attempts < max_attempts:
    connection_attempts += 1
    try:
        log_status("CONNECTION ATTEMPT", f"Attempt {connection_attempts} of {max_attempts}")
        
        fl.client.start_numpy_client(
            server_address="127.0.0.1:8081",
            client=MedicalFlashcardsClient(),
            transport="grpc-bidi",
            grpc_max_message_length=GRPC_MAX_MESSAGE_LENGTH
        )
        
        log_status("TRAINING SUCCESS", "Client completed all training rounds")
        break
        
    except ConnectionRefusedError:
        log_status("CONNECTION REFUSED", 
                  f"Server not available (attempt {connection_attempts})")
        if connection_attempts < max_attempts:
            logger.info(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    
    except Exception as e:
        log_status("CONNECTION ERROR", 
                  f"Error on attempt {connection_attempts}: {str(e)}")
        if connection_attempts < max_attempts:
            logger.info(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)

2025-03-09 15:13:43,360 - Client_1 - INFO - 
==================== CONNECTION SETUP ====================
2025-03-09 15:13:43,361 - Client_1 - INFO - Starting Flower client 1
Server address: 127.0.0.1:8081
2025-03-09 15:13:43,361 - Client_1 - INFO - ===========================================================
2025-03-09 15:13:43,363 - Client_1 - INFO - 
==================== CONNECTION ATTEMPT ====================
2025-03-09 15:13:43,364 - Client_1 - INFO - Attempt 1 of 3
2025-03-09 15:13:43,364 - Client_1 - INFO - =============================================================
2025-03-09 15:13:43,366 - Client_1 - INFO - 
==================== CLIENT INITIALIZATION ====================
2025-03-09 15:13:43,366 - Client_1 - INFO - Client 1 initializing...
2025-03-09 15:13:43,367 - Client_1 - INFO - ================================================================
2025-03-09 15:13:43,376 - Client_1 - INFO - 
==================== CLIENT READY ====================
2025-03-09 15:13:43,377 - Client_1


==================== CONNECTION SETUP ====================
Starting Flower client 1
Server address: 127.0.0.1:8081

==================== CONNECTION ATTEMPT ====================
Attempt 1 of 3

==================== CLIENT INITIALIZATION ====================
Client 1 initializing...

==================== CLIENT READY ====================
Client 1 initialized and ready

==================== PARAMETER RETRIEVAL ====================
Client 1: Getting parameters


INFO :      Sent reply
2025-03-09 15:13:44,361 - flwr - INFO - Sent reply
INFO :      
2025-03-09 15:13:58,690 - flwr - INFO - 
INFO :      Received: train message 08acf692-e783-4d4c-8a3d-3c054b443e5b
2025-03-09 15:13:58,699 - flwr - INFO - Received: train message 08acf692-e783-4d4c-8a3d-3c054b443e5b
2025-03-09 15:13:59,005 - Client_1 - INFO - 
==================== TRAINING START ====================
2025-03-09 15:13:59,006 - Client_1 - INFO - Client 1: Starting training round
2025-03-09 15:13:59,007 - Client_1 - INFO - =========================================================
2025-03-09 15:13:59,008 - Client_1 - INFO - 
==================== PARAMETER UPDATE ====================
2025-03-09 15:13:59,008 - Client_1 - INFO - Client 1: Setting parameters
2025-03-09 15:13:59,009 - Client_1 - INFO - ===========================================================



==================== TRAINING START ====================
Client 1: Starting training round

==================== PARAMETER UPDATE ====================
Client 1: Setting parameters


  0%|          | 0/186 [00:00<?, ?it/s]

...

{'train_runtime': 244.7678, 'train_samples_per_second': 12.257, 'train_steps_per_second': 0.76, 'train_loss': 0.9999844335740612, 'epoch': 2.98}

==================== TRAINING COMPLETE ====================
Client 1: Completed training round

==================== PARAMETER RETRIEVAL ====================
Client 1: Getting parameters


INFO :      Sent reply
2025-03-09 15:30:22,351 - flwr - INFO - Sent reply
INFO :      
2025-03-09 15:30:37,919 - flwr - INFO - 
INFO :      Received: evaluate message 6bab8ccc-e495-4b15-9ecf-efb6af67f242
2025-03-09 15:30:37,920 - flwr - INFO - Received: evaluate message 6bab8ccc-e495-4b15-9ecf-efb6af67f242
2025-03-09 15:30:38,132 - Client_1 - INFO - 
==================== EVALUATION ====================
2025-03-09 15:30:38,132 - Client_1 - INFO - Client 1: Evaluating model
2025-03-09 15:30:38,133 - Client_1 - INFO - =====================================================
2025-03-09 15:30:38,134 - Client_1 - INFO - 
==================== PARAMETER UPDATE ====================
2025-03-09 15:30:38,134 - Client_1 - INFO - Client 1: Setting parameters
2025-03-09 15:30:38,136 - Client_1 - INFO - ===========================================================



==================== EVALUATION ====================
Client 1: Evaluating model

==================== PARAMETER UPDATE ====================
Client 1: Setting parameters


ERROR :     Client raised an exception.
Traceback (most recent call last):
  File "c:\Users\Nil Atabey\anaconda3\Lib\site-packages\flwr\client\app.py", line 570, in start_client_internal
    reply_message = client_app(message=message, context=context)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nil Atabey\anaconda3\Lib\site-packages\flwr\client\client_app.py", line 143, in __call__
    return self._call(message, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nil Atabey\anaconda3\Lib\site-packages\flwr\client\client_app.py", line 126, in ffn
    out_message = handle_legacy_message_from_msgtype(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nil Atabey\anaconda3\Lib\site-packages\flwr\client\message_handler\message_handler.py", line 135, in handle_legacy_message_from_msgtype
    evaluate_res = maybe_call_evaluate(
                   ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nil Atabey\anaconda3\Lib\site-packages


==================== CONNECTION ERROR ====================
Error on attempt 3: Trainer: evaluation requires an eval_dataset.


In [41]:
# Test after training
log_status("POST-TRAINING EVALUATION", "Testing model after training...")
after_responses = evaluate_model_responses("After")

# Compare responses
comparison_data = {
    "client_id": CLIENT_ID,
    "timestamp": datetime.now().isoformat(),
    "connection_info": {
        "attempts": connection_attempts,
        "max_attempts": max_attempts,
        "status": "success" if connection_attempts < max_attempts else "failed"
    },
    "comparisons": []
}

log_status("RESULTS COMPARISON", "Analyzing before/after performance")
before_similarities = []
after_similarities = []

for q, a in zip(test_questions, test_answers):
    before_sim = calculate_similarity(a, before_responses[q]['model_response'])
    after_sim = calculate_similarity(a, after_responses[q]['model_response'])
    before_similarities.append(before_sim)
    after_similarities.append(after_sim)
    
    comparison = {
        "question": q,
        "ground_truth": a,
        "before": before_responses[q]['model_response'],
        "after": after_responses[q]['model_response'],
        "similarity_before": before_sim,
        "similarity_after": after_sim,
        "improvement": after_sim - before_sim
    }
    comparison_data["comparisons"].append(comparison)
    
    print(f"Improvement: {(after_sim - before_sim) * 100:.1f}%")

2025-03-09 15:32:23,944 - Client_1 - INFO - 
==================== POST-TRAINING EVALUATION ====================
2025-03-09 15:32:23,946 - Client_1 - INFO - Testing model after training...
2025-03-09 15:32:23,947 - Client_1 - INFO - ===================================================================
2025-03-09 15:32:23,947 - Client_1 - INFO - 
==================== AFTER TRAINING EVALUATION ====================
2025-03-09 15:32:23,948 - Client_1 - INFO - Starting model evaluation...
2025-03-09 15:32:23,949 - Client_1 - INFO - ====================================================================
c:\Users\Nil Atabey\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may obser


==================== POST-TRAINING EVALUATION ====================
Testing model after training...

==================== AFTER TRAINING EVALUATION ====================
Starting model evaluation...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Model Response: Question: What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?
Answer: Very low Pthal levels are associated with very high Ca+ level levels.


Question, what is CaII+ and what are the two types of Ca-II levels that are commonly associated in relation to very Low MG2+.
A: CaI+ is a measurement of how high the levels of calcium in the body. This is often associated
Ground Truth: Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.
--------------------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


Question: What is Plummer-Vinson syndrome and what are the main symptoms associated with this condition?
Model Response: Question: What is Plummer-Vinson syndrome and what are the main symptoms associated with this condition?
Answer: Plumpy-Walker syndrome is a condition characterized by the presence of a triad of symptoms: acute (acute), chronic (coagulary), and/or delayed (intermediate). These symptoms can occur in various stages of the disease, and can range from mild to severe. The severity of these symptoms is not always known, but it is possible to have multiple episodes
Ground Truth: Plummer-Vinson syndrome is a medical condition characterized by iron deficiency anemia, esophageal webs, and atrophic glossitis. Some common symptoms of this syndrome include difficulty swallowing, mouth sores, and fatigue.
--------------------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question: What is the genetic cause of cystic fibrosis, and which gene and chromosome is affected by this defect?
Model Response: Question: What is the genetic cause of cystic fibrosis, and which gene and chromosome is affected by this defect?
Answer: Cystic fibrillation (fibrils that form a chain of DNA defects) is caused by a defect in the CFTR gene on chromosome 5q. This gene is involved in regulating cell growth and division, which results in a range of symptoms including cystitis, malabsorption, pancreatic cysts, lung damage, liver damage and rupture, among
Ground Truth: Cystic fibrosis is caused by a genetic defect in the CFTR gene, which is located on chromosome 7. This gene is responsible for producing a protein that regulates the movement of salt and water in and out of cells. When the CFTR gene is defective, it leads to the production of thick, sticky mucus in the lungs, pancreas, and other organs, which can cause a range of symptoms and complications associated with cystic 

2025-03-09 15:32:34,154 - Client_1 - INFO - 
==================== RESULTS COMPARISON ====================
2025-03-09 15:32:34,155 - Client_1 - INFO - Analyzing before/after performance
2025-03-09 15:32:34,155 - Client_1 - INFO - =============================================================



Question: What is the relationship between upper motoneuron lesions and paralysis?
Model Response: Question: What is the relationship between upper motoneuron lesions and paralysis?
Answer: Upper motonesurs are associated with paralysis due to the presence of a thickening of the corpus callosum.


Question is: How is upper motor neuron paralysis caused?

Answer, Upper motor neurons are caused by the degeneration of dopaminergic neurons in the substantia nigra of each muscle of an elephant's body. This degenerated neuron leads to a loss of sensation and movement in both
Ground Truth: Upper motoneuron lesions may cause spastic paralysis.
--------------------------------------------------

==================== RESULTS COMPARISON ====================
Analyzing before/after performance
Improvement: 7.5%
Improvement: 0.7%
Improvement: -0.2%
Improvement: 0.6%
Improvement: -1.9%


In [42]:
# Calculate overall metrics
avg_improvement = np.mean(np.array(after_similarities) - np.array(before_similarities))
comparison_data["metrics"] = {
    "average_similarity_before": float(np.mean(before_similarities)),
    "average_similarity_after": float(np.mean(after_similarities)),
    "average_improvement": float(avg_improvement),
    "max_improvement": float(np.max(np.array(after_similarities) - np.array(before_similarities))),
    "min_improvement": float(np.min(np.array(after_similarities) - np.array(before_similarities)))
}

In [44]:
# Save results
results_path = os.path.join(temp_dir, f"client_{CLIENT_ID}_results.json")
with open(results_path, "w") as f:
    json.dump(comparison_data, f, indent=2)

log_status("FINAL STATISTICS", 
         f"Client {CLIENT_ID} Training Summary\n"
         f"Total examples: {len(small_dataset)}\n"
         f"Device: {device}\n"
         f"Model parameters: {sum(p.numel() for p in model.parameters())}\n"
         f"Average improvement: {avg_improvement * 100:.1f}%\n"
         f"Results saved: {results_path}")

# Save final model
save_model()

2025-03-09 15:33:58,811 - Client_1 - INFO - 
==================== FINAL STATISTICS ====================
2025-03-09 15:33:58,812 - Client_1 - INFO - Client 1 Training Summary
Total examples: 1000
Device: cuda
Model parameters: 124439808
Average improvement: 1.3%
Results saved: C:\Users\NILATA~1\AppData\Local\Temp\tmp9bgt6qn4\client_1_results.json
2025-03-09 15:33:58,813 - Client_1 - INFO - ===========================================================



==================== FINAL STATISTICS ====================
Client 1 Training Summary
Total examples: 1000
Device: cuda
Model parameters: 124439808
Average improvement: 1.3%
Results saved: C:\Users\NILATA~1\AppData\Local\Temp\tmp9bgt6qn4\client_1_results.json


2025-03-09 15:33:59,816 - Client_1 - INFO - 
==================== MODEL SAVED ====================
2025-03-09 15:33:59,816 - Client_1 - INFO - Model saved to C:\Users\NILATA~1\AppData\Local\Temp\tmp9bgt6qn4\medical-model-client-1
2025-03-09 15:33:59,818 - Client_1 - INFO - ======================================================



==================== MODEL SAVED ====================
Model saved to C:\Users\NILATA~1\AppData\Local\Temp\tmp9bgt6qn4\medical-model-client-1
